In [2]:
!pip install tensorflow==2.15

In [5]:
import tensorflow as tf

In [6]:
tf.__version__

'2.15.0'

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [8]:
!kaggle datasets download -d saumyamohandas/animal-dataset


Dataset URL: https://www.kaggle.com/datasets/saumyamohandas/animal-dataset
License(s): unknown
 79% 49.0M/62.3M [00:00<00:00, 63.8MB/s]
100% 62.3M/62.3M [00:00<00:00, 78.1MB/s]


In [9]:



!unzip '/content/animal-dataset.zip'

Archive:  /content/animal-dataset.zip
  inflating: dataset/Testing/bears/k4 (100).jpeg  
  inflating: dataset/Testing/bears/k4 (100).jpg  
  inflating: dataset/Testing/bears/k4 (101).jpeg  
  inflating: dataset/Testing/bears/k4 (101).jpg  
  inflating: dataset/Testing/bears/k4 (102).jpeg  
  inflating: dataset/Testing/bears/k4 (102).jpg  
  inflating: dataset/Testing/bears/k4 (103).jpeg  
  inflating: dataset/Testing/bears/k4 (104).jpeg  
  inflating: dataset/Testing/bears/k4 (105).jpeg  
  inflating: dataset/Testing/bears/k4 (106).jpeg  
  inflating: dataset/Testing/bears/k4 (107).jpeg  
  inflating: dataset/Testing/bears/k4 (108).jpeg  
  inflating: dataset/Testing/bears/k4 (109).jpeg  
  inflating: dataset/Testing/bears/k4 (110).jpeg  
  inflating: dataset/Testing/bears/k4 (71).jpg  
  inflating: dataset/Testing/bears/k4 (72).jpeg  
  inflating: dataset/Testing/bears/k4 (72).jpg  
  inflating: dataset/Testing/bears/k4 (73).jpeg  
  inflating: dataset/Testing/bears/k4 (73).jpg  
  in

In [10]:
trainPath = '/content/dataset/Training'
testPath = '/content/dataset/Testing'

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
train = train_datagen.flow_from_directory(trainPath,target_size=(224,224),batch_size=16)
test = test_datagen.flow_from_directory(testPath,target_size=(224,224),batch_size=16)

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


# VGG16

In [14]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [16]:
vgg = VGG16(include_top=False,input_shape=(224,224,3))

In [17]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
for layer in vgg.layers:
  print(layer)

In [19]:
for layer in vgg.layers:
  layer.trainable=False

In [20]:
x = Flatten()(vgg.output)

In [21]:
output = Dense(4,activation='softmax')(x)

In [22]:
vgg16 = Model(vgg.input,output)

In [23]:
vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
vgg16.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [26]:
vgg16.fit(train,validation_data=test,epochs=2)

Epoch 1/2
78/78 [==============================] - 18s 236ms/step - loss: 0.0530 - accuracy: 0.9927 - val_loss: 0.0196 - val_accuracy: 0.9969
Epoch 2/2
78/78 [==============================] - 19s 237ms/step - loss: 0.0372 - accuracy: 0.9943 - val_loss: 0.0108 - val_accuracy: 0.9969


# Resnet

In [27]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [28]:
resnet50 = ResNet50(include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [29]:
for layer in resnet50.layers:
  print(layer)

In [30]:
for layer in resnet50.layers:
  layer.trainable=False

In [31]:
x = Flatten()(resnet50.output)

In [32]:
output = Dense(4,activation='softmax')(x)

In [33]:
resnet50 = Model(resnet50.input,output)

In [34]:
resnet50.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [35]:
resnet50.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
resnet50.fit(train,validation_data=test,epochs=10)

Epoch 1/20
78/78 [==============================] - 25s 256ms/step - loss: 2.5801 - accuracy: 0.3893 - val_loss: 0.9471 - val_accuracy: 0.5491
Epoch 2/20
78/78 [==============================] - 18s 228ms/step - loss: 1.3741 - accuracy: 0.5024 - val_loss: 1.0056 - val_accuracy: 0.5859
Epoch 3/20
78/78 [==============================] - 19s 241ms/step - loss: 1.1440 - accuracy: 0.5574 - val_loss: 0.9589 - val_accuracy: 0.6227
Epoch 4/20
78/78 [==============================] - 18s 229ms/step - loss: 1.3989 - accuracy: 0.5380 - val_loss: 0.7455 - val_accuracy: 0.7209
Epoch 5/20
78/78 [==============================] - 20s 255ms/step - loss: 1.4588 - accuracy: 0.5468 - val_loss: 0.6755 - val_accuracy: 0.7270
Epoch 6/20
78/78 [==============================] - 18s 229ms/step - loss: 1.0789 - accuracy: 0.6155 - val_loss: 1.5109 - val_accuracy: 0.5460
Epoch 7/20
78/78 [==============================] - 19s 243ms/step - loss: 0.9824 - accuracy: 0.6478 - val_loss: 1.1300 - val_accuracy: 0.6380

KeyboardInterrupt: 

# Inception

In [ ]:
#!pip install tensorflow==2.13

In [37]:
train = train_datagen.flow_from_directory(trainPath,target_size=(299,299),batch_size=16)
test = test_datagen.flow_from_directory(testPath,target_size=(299,299),batch_size=16)

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


In [38]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [39]:
InceptionV3 = InceptionV3(include_top=False,input_shape=(299 ,299,3))

87910968/87910968 [==============================] - 0s 0us/step


In [42]:

for layer in InceptionV3.layers:
  print(layer)

In [43]:
x = Flatten()(InceptionV3.output)

In [44]:
output = Dense(4,activation='softmax')(x)

In [45]:
InceptionV3 = Model(InceptionV3.input,output)

In [46]:
InceptionV3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_4[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0]

In [47]:
InceptionV3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [48]:
InceptionV3.fit(train,validation_data=test,epochs=5)

Epoch 1/5
78/78 [==============================] - 80s 504ms/step - loss: 2.3901 - accuracy: 0.6341 - val_loss: 24.2852 - val_accuracy: 0.3282
Epoch 2/5
78/78 [==============================] - 34s 430ms/step - loss: 1.0731 - accuracy: 0.7359 - val_loss: 486.4543 - val_accuracy: 0.4264
Epoch 3/5
78/78 [==============================] - 33s 413ms/step - loss: 0.8573 - accuracy: 0.7779 - val_loss: 1857.3807 - val_accuracy: 0.3098
Epoch 4/5
78/78 [==============================] - 34s 438ms/step - loss: 0.6972 - accuracy: 0.8199 - val_loss: 1.2710 - val_accuracy: 0.6626
Epoch 5/5
78/78 [==============================] - 32s 414ms/step - loss: 0.5536 - accuracy: 0.8635 - val_loss: 20.8734 - val_accuracy: 0.3528


# Xception

In [49]:
train = train_datagen.flow_from_directory(trainPath,target_size=(299,299),batch_size=16)
test = test_datagen.flow_from_directory(testPath,target_size=(299,299),batch_size=16)

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


In [51]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [52]:
Xception = Xception(include_top=False,input_shape=(299,299,3))

83683744/83683744 [==============================] - 0s 0us/step


In [55]:
x = Flatten()(Xception.output)

In [56]:
output = Dense(4,activation='softmax')(x)

In [57]:
Xception = Model(Xception.input,output)

In [58]:
Xception.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_5[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'

In [59]:
Xception.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [60]:
Xception.fit(train,validation_data=test,epochs=2)

Epoch 1/2
78/78 [==============================] - 81s 653ms/step - loss: 0.7135 - accuracy: 0.8675 - val_loss: 2.0384 - val_accuracy: 0.8344
Epoch 2/2
78/78 [==============================] - 41s 525ms/step - loss: 0.2606 - accuracy: 0.9354 - val_loss: 0.5612 - val_accuracy: 0.9080
